In [82]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score

# collect data
fileDir = "../Data/DataForClassification/"
fileList = os.listdir(fileDir)
print(fileList)

# # auto method that go through all the file in directory
# for file in fileList:
#     if not file.startswith('.'):
#         if file.endswith(".txt"):
#             file = file[:-4]

# hard code to read the file one by one
author0 = []
author1 = []
with open(fileDir+"lei wang0.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.strip().split("\t")
        author0.append(read_data[1].split(" "))

with open(fileDir+"lei wang1.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.strip().split("\t")
        author1.append(read_data[1].split(" "))
print(author0[0])
print(author1[0])

['chung-may yang1.txt', 'david g lloyd0.txt', 'david g lloyd1.txt', 'jeong hwan kim1.txt', 'chung-may yang0.txt', 'michael wagner0.txt', 'kevin m. ryan0.txt', 'michael wagner1.txt', 'lei wang0.txt', 'jeong hwan kim0.txt', 'lei wang1.txt', 'kevin m. ryan1.txt']
['-0.232623', '-0.12317', '-0.178615', '-0.120828', '-0.0637316', '0.119655', '-0.136511', '-0.012849', '-0.125184', '0.0461908', '0.0581749', '-0.183963', '-0.0459688', '0.19452', '-0.0170126', '-0.142147', '0.066843', '0.0642291', '0.00959536', '-0.107407', '0.0097539', '0.0416746', '-0.103548', '0.0770354', '0.108264', '0.0566145', '-0.0668466', '0.163799', '0.0729433', '-0.000337214', '-0.161726', '0.046857', '0.0794689', '-0.0573545', '-0.035842', '-0.0549588', '0.0425291', '-0.0653112', '0.0527278', '0.0105899', '-0.15386', '-0.104491', '-0.0575352', '-0.113251', '-0.0697307', '0.0515379', '-0.0936607', '0.0598422', '-0.0820808', '-0.0636461', '0.0354209', '-0.0147456', '0.119361', '-0.0330736', '0.056727', '-0.165324', '-0

In [83]:
# size of each class
print(len(author0))
print(len(author1))

53
64


In [66]:
# number of features (dimension)
print(len(author0[0]))
print(len(author1[1]))

100
100


In [84]:
# reconstract data so that we can feed it to svm
import pandas as pd
classOne = pd.DataFrame(author0)
classOne["label"] = 0
#print(classOne[:2:])
classTwo = pd.DataFrame(author1)
classTwo["label"] = 1
#print(classTwo[:2:])
# combine data from different class get all data
combinedData = pd.concat([classOne, classTwo])
combinedData = combinedData.sample(frac=1).reset_index(drop=True)
print(combinedData[:5])
# split data and label
data = combinedData.drop('label', axis=1)
label = combinedData['label']

           0           1           2           3           4          5  \
0  0.0641123  -0.0615801   -0.251977   -0.023063  -0.0188361   0.038366   
1  -0.233405  -0.0512902   -0.219455   -0.023153   -0.084073  0.0945192   
2  -0.202214   -0.126435  -0.0815735  -0.0469542  -0.0273222  0.0569356   
3  -0.253184   0.0318314   -0.248105   -0.184714    0.166186  0.0543164   
4  -0.153682  -0.0443485   -0.107922   -0.139944  -0.0898169   0.055224   

            6           7           8            9  ...           91  \
0  -0.0937673  -0.0578285   -0.100574    0.0223806  ...   0.00321343   
1  -0.0710282  -0.0337651  -0.0476832     0.108345  ...    0.0826303   
2   -0.076542  -0.0407632   -0.105154  -0.00680006  ...    0.0907763   
3   0.0603233   0.0190748  -0.0815832     0.092431  ...    0.0230457   
4   -0.193431   -0.130026  -0.0899581    0.0777159  ...   -0.0151897   

           92          93           94          95          96          97  \
0  -0.0127611   -0.106231   -0.0856751

In [60]:
# cross validation
def k_fold_cv(data, label, classifier, dataname):
    kf = KFold(n_splits=10, shuffle=True)
    # create lists to collect statistic
    tp = []
    fp = []
    tn = []
    fn = []
    for train_index, test_index in kf.split(data):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        label_train, test_true_label = label.iloc[train_index], label.iloc[test_index]
        # fit data to svm
        classifier.fit(data_train, label_train)
        # get predicted label
        label_pred = classifier.predict(data_test)
        # find round confusion matrix
        round_tn, round_fp, round_fn, round_tp = metrics.confusion_matrix(test_true_label, label_pred).ravel()
        # add data data to array
        tp.append(round_tp)
        fp.append(round_fp)
        fn.append(round_fn)
        tn.append(round_tn)
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    print("Dataset: {name}\nTrue positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
          .format(name=dataname, tp=np.sum(tp), fp=np.sum(fp), fn=np.sum(fn), tn=np.sum(tn)))
    ppv, npv, specificity, sensitivity, accuracy = calculate_important_value(np.sum(tp), np.sum(tn),
                                                                             np.sum(fp), np.sum(fn), len(data))
    return ppv, npv, specificity, sensitivity, accuracy

In [53]:
# calculate ppv,npv,specificity,sensitivity, and accuracy
def calculate_important_value(tp, tn, fp, fn, sample_length):
    # 1. Positive predicted value (PPV) or precision aka hit rate = True positive/ )True positive + False positive)
    ppv = (tp / (tp + fp))
    # 2. Negative predicted value (NPV) = True negative / (True negative + False negative)
    npv = (tn / (tn + fn))
    # 3. Specificity = (1 - False positive)
    specificity = (tn / (tn + fp))
    # 4. Sensitivity = True positive
    sensitivity = (tp / (tp + fn))
    # 5. Accuracy = (True positive + True negative) / Total number of sample
    accuracy = (tp + tn) / sample_length
    print('PPV: ', ppv)
    print('NPV: ', npv)
    print('Specificity: ', specificity)
    print('Sensitivity: ', sensitivity)
    print('Accuracy: ', accuracy,)
    return ppv, npv, specificity, sensitivity, accuracy

In [89]:
# # create linear SVM model
# linear_svc = svm.SVC(kernel='linear')
# print(linear_svc)

# create rbf SVM model
rbf_svc = svm.SVC(kernel='rbf', C=100)
print(rbf_svc)

# fit model and do 10-fold cv
k_fold_cv(data, label, rbf_svc, "Same name author")


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Dataset: Same name author
True positive: 63, False positive: 0, False negative: 1, True negative: 53
PPV:  1.0
NPV:  0.9814814814814815
Specificity:  1.0
Sensitivity:  0.984375
Accuracy:  0.9914529914529915


(1.0, 0.9814814814814815, 1.0, 0.984375, 0.9914529914529915)